# *Introduction*

With this project we aim to find a minimum-length solution of a configuration G
of the Yashi game, if it exists, using a SAT encoding.

An instance of the Yashi game is specifid by a $n\times n$ integer grid for some $n>2$, on which $p>2$ nodes are placed. A solution of the game consists in drawing horizontal and vertical segments, satisfying the following conditions:


1.   No two segments cross each other;
2.   The segments form a tree, i.e. they form a graph without cycles. Put differently, for every two nodes a and b there is exactly one path between a and b.

We'll start by defining some functions for the creation of hard and soft clauses and then we'll see some examples of the finding of a solution.

# *Import of the libraries*

We import some libraries that we are going to use in the definition of the functions.

In [ ]:
!pip install python-sat

In [ ]:
import random
import matplotlib.pyplot as plt
from itertools import combinations
from pysat.examples.rc2 import RC2
from pysat.formula import WCNF
random.seed(124)

# Definition of Functions

## *Initial setting*

At first, we create a grid $n\times n$ with $n_{points}$, where both $n$ and $n_{points}$ are decided by the player, based on the difficulty of the game. The $n_{points}$ are generated randomly on the grid.

In [ ]:
def random_start(n=2, n_points=2):
  # Create a list of random points
  unordered_points = []
  while len(unordered_points) < n_points:
      x = random.randint(1, n-1)
      y = random.randint(1, n-1)
      if (x,y) not in unordered_points:
        unordered_points.append((x, y))
  points = sorted(unordered_points, key=lambda x: (x[0], x[1]))
  # Create a list of the vertical and horizontal links between the points
  links = []
  for i in range(len(points) - 1):
    k=0
    h=0
    for j in range(i + 1, len(points)):
      if points[i][0] == points[j][0] and h < 1:
        links.append(((points[i][0], points[i][1]),(points[j][0],points[j][1]))) #we insert the horizontal links between two points
        h = 1  # this number guardantees that one point is linked only to the closest one
      elif points[i][1] == points[j][1] and k < 1:
        links.append(((points[i][0], points[i][1]),(points[j][0],points[j][1]))) #we insert the vertical links between two points
        k = 1
  return points, links



In [ ]:
def graphical_implementation(n, initial_points):
    # Create a figure and axis
    fig, ax = plt.subplots()

    # Plot vertical grid lines
    for i in range(n + 1):
        ax.axvline(x = i, color = '#C1CDCD', linestyle = '--', linewidth = 1)

    # Plot horizontal grid lines
    for i in range(n + 1):
        ax.axhline(y = i, color = '#C1CDCD', linestyle = '--', linewidth = 1)

    # Set axis limits and labels
    ax.set_xlim(0, n)
    ax.set_ylim(0, n)
    ax.set_xticks(range(n + 1))
    ax.set_yticks(range(n + 1))


    # Remove axis labels
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    # Display the grid
    plt.grid(True)

    for point in initial_points:
        x, y = point
        ax.scatter(x, y, color = 'black', marker = 'o')

    plt.show()

## *Creation of variables*

We then create a function that gives a unique index to each segment that links two points that can be linked (two points that are on the same horizontal or vertical line and that are the closest to each other). In this way we are going to create the clauses that will tell if the $i$-th link is present or not in the solution (e.g. if we have -1 it means that the first link will not be present in the solution).   

In order to go back from the index to the link, we define also the inverse function that, given an index, returns the link that it refers to. We are going to use this for the graphical implementation of the solution.

In [ ]:
# we give an index at each link that we have in the links list

def v(edge, links):
  if edge in links:
    index = links.index(edge)
  else:
    edge = (edge[1], edge[0])
    index = links.index(edge)
  return index + 1

In [ ]:
# we define also the inverse function in order to draw the right links at the end

def find_link(index, links):
  for edge in links:
    if v(edge, links) == index:
      return edge

## *No crossing links*

We define a function that creates a list of clauses in order to not have that two links cross each other. This is an hard kind of clause.

Therefore, if we have for example that the links 1 and 7 cross each other, we will have the clause

 $$\neg{link_1} \lor \neg{link_7}$$

In [ ]:
#we define the constraints if we have that two links are crossing. In that case we cannot have them both.
def vectors_crossing_constraints(links):
  clauses=[]
  for i in range(len(links)-1): #example
    for j in range(i + 1, len(links)):
      if links[i][0][1] == links[i][1][1] and links[j][0][0] > links[i][0][0] and links[j][0][0] < links[i][1][0] and links[j][0][1]< links[i][0][1] and links[j][1][1]>links[i][0][1]:
        clauses.append([-v(links[i],links), -v(links[j],links)])
  return clauses


## *No cycles*
We define a function that creates a list of clauses in order to not have links that create a cycle. Those are hard clauses.

For example, if the links 1,5,6,7 and 8 create a cycle, the clause will be:

$$\neg{link_1} \lor \neg{link_5} \lor \neg{link_6} \lor \neg{link_7} \lor \neg{link_8} $$

In order to facilitate the definition of this function, we previously create the graph representation of our links.

In [ ]:
# we define the constraints for a cycle
# first we need to create a graph representation:

def graph_representation(links, points):
  graph = {}
  for link in links:
    a, b = link
    if a not in graph:
        graph[a] = []
    graph[a].append(b)
    if b not in graph:
        graph[b] = []
    graph[b].append(a)
  if len(graph) < len(points):
    points_in_graph=[i for i in graph]
    solitary_points = [x for x in points if x not in points_in_graph]
    for i in solitary_points:
      graph[i]=[]
  return graph


In [ ]:
# we then define a function that finds all the cycles that are present in the yashi configuration
def find_all_cycles(graph):
    def dfs(node, visited, stack):
        visited[node] = True
        stack.append(node)
        if node in graph:
            for neighbor in graph[node]:
                if not visited[neighbor]:
                    dfs(neighbor, visited, stack)
                elif neighbor in stack:
                    cycle = stack[stack.index(neighbor):]
                    if len(cycle)>=3:
                      edges = [(cycle[i], cycle[i+1]) for i in range(len(cycle)-1)]
                      edges = edges + [(cycle[0], cycle[-1])]
                      cycle_reordered = sorted(cycle, key=lambda x: (x[0], x[1]))
                      if cycle_reordered not in cycles:
                        edge.append(edges)
                        cycles.append(cycle_reordered)


        stack.pop()
        visited[node] = False

    visited = {node: False for node in graph}
    cycles = []
    edge = []

    for node in graph:
        if not visited[node]:
            dfs(node, visited, [])
    edges_sorted=[]
    for i in range(len(edge)):
        edges_sorted.append([])
        for j in range(len(edge[i])):
          new_tuple= sorted(edge[i][j], key=lambda x: (x[0], x[1]))
          edges_sorted[i].append(tuple(new_tuple))
    edges_sorted
    return edges_sorted

In [ ]:
# we now create the constraints relative to the cycles that are present:
def cycles_constraints(links):
  clauses=[]
  graph = graph_representation(links, points)
  cycles = find_all_cycles(graph)
  for e in cycles:
    list_i=[]
    for i in e:
      list_i.append(-v(i, links))
    clauses.append(list_i)
  return clauses

## *No isolation*

We define a function that creates a list of clauses in order to not have any point that is not linked to any other point.

If a point is isolated, we add the constraint

 $$\neg{link_1} \land {link_1} $$

  that is never satisfied, so the SAT solver won't find any solution.

In [ ]:
def constraints_isolation(links):
  clauses = []
  graph = graph_representation(links,points)
  for i in graph:
    if len(graph[i]) == 0:  #in this way it is impossible to satisfy the constraint
      clauses.append([-1])
      clauses.append([1])
    else:
      clauses.append([v((i, graph[i][j]), links) for j in range(len(graph[i]))])

  return clauses



## *Exactly $n_{points} - 1$ links*

With the following function we create a list of hard clauses that guarantees that exactly $n_{points} - 1$ links are present in the final solution.
This function is the combination of having at most k links and at least k links.

In order to do so, we previously define a function that finds all the combinations (without significance of the order) of k elements among n elements.

In [ ]:
#we define a function that finds all the combination of k numbers among n numbers, n>k
def find_combinations(sequence, k):
    # Ensure k is not greater than the length of the sequence
    if k > len(sequence):
        raise ValueError("k cannot be greater than the length of the sequence.")
        return None
    # Use itertools.combinations to find all combinations of k numbers
    combinations_list = list(combinations(sequence, k))

    return combinations_list

In [ ]:
def exactly_l_links(links):
  clauses = []
  sequence = list(range(1, len(links) + 1))
  if len(sequence) <= len(points) - 1: #in this case I want all the links
    clauses.append([i for i in sequence])
  else:
    combination_at_least = find_combinations(sequence, len(links) - (len(points)-1) + 1 )
    for i in combination_at_least:
      clauses.append([j for j in i])
    combination_at_most = find_combinations(sequence, len(points))
    for i in combination_at_most:
      clauses.append([-j for j in i])
  return clauses


## *Weights of the links*

In order to find the solution with minimum length, we need to associate some weights to each link and we decided to give to each link the weight that is its length, with a minus. This is because with the maxSAT solver that we are going to use, the solution that is chosen is the one that leaves out the links with the minimum weight. Instead, in this case we want to choose the links that have less length, and this is the reason of the presence of the minus sign.

With this function we create a list of soft clauses.

For example, if the lenght of the third link is 4 and the length of the fifth link is 1, we create the clauses:

$$[link_3, weight=-4]$$
$$[link_5, weight=-1]$$

In this way, if the solver has to choose between the two links, it will keep the $link_5$ that has length 1, since if it leaves out the $link_3$ it will have a minor weight.

In [ ]:
# We want to associate to each link the weight of its length, in order to minimize the length of the problem when the link is present
#rc2.add_clause([1], weight=1)

def weights_links(links):
  clauses_weights = []
  for edge in links:
    clauses_weights.append(([v(edge,links)], -abs(edge[0][0] - edge[1][0] + edge[0][1] - edge[1][1] )))
  return clauses_weights


## *Yashi configuration solution*

With this function we implement a maxSAT solver that uses all the clauses that were created with the previous functions to find a solution of the Yashi configuration. If the configuration has no solution, it returns an error.

We also define a function that creates the graphical representation of the solution.


In [ ]:
def solver_Yashi_configuration(links):
  clauses_crossing = vectors_crossing_constraints(links)
  clauses_cycles = cycles_constraints(links)
  clauses_isolation = constraints_isolation(links)
  clauses_n_links = exactly_l_links(links)
  hard_clauses = clauses_crossing + clauses_cycles + clauses_isolation + clauses_n_links

  soft_clauses = weights_links(links)

  rc2 = RC2(WCNF())
  for i in hard_clauses:
    rc2.add_clause(i)
  for j in soft_clauses:
    rc2.add_clause(j[0], j[1])

  model = rc2.compute()

  if model == None:
    print('Error: this configuration of the Yashi game has no solution. Try another configuration!')
    return None

  return model

In [ ]:
def graphical_implementation_of_solution(n, initial_points, index, links):
    if index == None:
      print( 'There is no solution, hence we cannot represent it')
      return None
    # Create a figure and axis
    fig, ax = plt.subplots()

    # Plot vertical grid lines
    for i in range(n + 1):
        ax.axvline(x=i, color='#C1CDCD', linestyle='--', linewidth=1)

    # Plot horizontal grid lines
    for i in range(n + 1):
        ax.axhline(y=i, color='#C1CDCD', linestyle='--', linewidth=1)

    # Set axis limits and labels
    ax.set_xlim(0, n)
    ax.set_ylim(0, n)
    ax.set_xticks(range(n + 1))
    ax.set_yticks(range(n + 1))


    # Remove axis labels
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    # Display the grid
    plt.grid(True)

    for point in initial_points:
        x, y = point
        ax.scatter(x, y, color='black', marker='o')

    links_present = []
    for i in index:
      if i > 0:
        l = find_link(i, links)
        links_present.append(l)

    for line in links_present:
        ax.plot([line[0][0], line[1][0]], [line[0][1], line[1][1]], color='#242424', linewidth=1.5)

    plt.show()

# *Example with a grid $6 \times 6$ and 11 points*

We try our algorithm to see if it works with a grid $6 \times 6$ and 11 points. We will first show the initial configuration with only the points, and then our final solution. We will set a random seed for recreation.

In [ ]:
random.seed(124)
points, links = random_start(n=6, n_points=11)
graphical_implementation(6, points)


In [ ]:
s=solver_Yashi_configuration(links)
graphical_implementation_of_solution(6, points, s, links)

# *Example with no solution*

We show also an example where there is no solution; in this case the reason is that there is one point that is not linked to any other point.

In [ ]:
random.seed(12)
points, links = random_start(n=6, n_points=11)

graphical_implementation(6, points)
s=solver_Yashi_configuration(links)
graphical_implementation_of_solution(6, points, s, links)



# *Game*

You can now customize the size of the grid and the number of points. Let's see if you can tell if a configuration of the Yashi game has or not a solution!

In [ ]:
random.seed(122)
n = int(input('Please insert the size of the grid (the number must be a positive integer bigger than 2): '))
n_points = int(input('Now insert the number of points you want to play with (the number must be a positive integer bigger than 2): '))

while n < 3:
  print("This number is too low, let's try with something else!")
  n = int(input('Please insert the size of the grid: '))

while n_points <= n or n_points > (n-1)**2:
   print("This number is too low or too high, let's try with something else!")
   n_points = int(input('Please insert the number of points you want to play with: '))

points, links = random_start(n, n_points)

print('This is the configuration you are going to play with.')
graphical_implementation(n, points)

answer = input('Can you tell me if there is a solution? Answer yes or no: ')

answer = answer.lower()
possible_answers = ['yes', 'no']
while answer not in possible_answers:
  print("I'm sorry, can you tell me again your answer? Please be sure to write just yes or no")
  answer = input('Your answer is: ')
  answer = answer.lower()

s = solver_Yashi_configuration(links)

if answer == 'yes':
  if s != None:
    print('You are right! There exist a solution for this configuration. What I have found is the solution with minimum length, but there could exist other solutions!')
    graphical_implementation_of_solution(n, points, s, links)
  else:
    print("I'm sorry :( There are no solution for this configuration of Yashi game. Try again")

if answer == 'no':
  if s != None:
    print("I'm sorry :( There is indeed at least a solution for this configuration of Yashi game. Try again!")
    print("A solution that I have found is the solution with minimum length.")
    graphical_implementation_of_solution(n, points, s, links)
  else:
    print('You are right! There exist no solution for this configuration.')
